In [2]:
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
from os.path import isfile, join

List of what to do:


1.   Blur before and after image using gaussian blur
2.   Merge 2 photos together
3.   Based on distance of moving objects, apply amount of mo




In [3]:
def gaussian_blur(x,y,st_dev):
  exponent_term = -((x ** 2) + (y ** 2))/(2 * st_dev ** 2)
  e_term = math.e ** exponent_term
  constant_term = 1/(2 * math.pi * st_dev ** 2)
  return e_term * constant_term

In [4]:
def neighbor_values_gauss(x,y,rgb,top_left,bottom_right,num_of_neigh,old_image,dev,no_ugly_box):

  #Getting all the values for each boundary to blur
  top = top_left[1]
  left = top_left[0]
  bottom = bottom_right[1]
  right = bottom_right[0]
  
  #to get rid of ugly box
  if no_ugly_box == 1:
    bottom = len(old_image)
    right = len(old_image[0])
    top = 0
    left = 0
    
  #Checking edge cases
  Col_Count = -num_of_neigh
  Row_Count = -num_of_neigh
  Col_Stop = num_of_neigh
  Row_Stop = num_of_neigh

  x_distance_left = x - left 
  if x_distance_left >= num_of_neigh:
    x_distance_left = num_of_neigh
  x_distance_right = right - x - 1
  if x_distance_right >= num_of_neigh:
    x_distance_right = num_of_neigh
  y_distance_top = y - top
  if y_distance_top >= num_of_neigh:
    y_distance_top = num_of_neigh
  y_distance_bottom = bottom - y - 1
  if y_distance_bottom >= num_of_neigh:
    y_distance_bottom = num_of_neigh

  Col_Count = Col_Count + (num_of_neigh - x_distance_left)
  Row_Count = Row_Count + (num_of_neigh - y_distance_top)
  Col_Stop = Col_Stop - (num_of_neigh - x_distance_right)
  Row_Stop = Row_Stop - (num_of_neigh - y_distance_bottom)

  #holds the final pixel value for that color
  final_pixel_value = 0

  for Row in range(Row_Count, Row_Stop + 1):
    for Col in range(Col_Count, Col_Stop + 1):
    
      #Summing up all neighboring pixel values based on gaussian distribution
      final_pixel_value = final_pixel_value + gaussian_blur(Col, Row, dev) * old_image[y+Row][x+Col][rgb]

  return final_pixel_value

In [5]:
def neighbor_values_box(x,y,rgb,top_left,bottom_right,num_of_neigh,old_image,no_ugly_box):

  #Getting all the values for each boundary to blur
  top = top_left[1]
  left = top_left[0]
  bottom = bottom_right[1]
  right = bottom_right[0]

  #to get rid of ugly box
  if no_ugly_box == 1:
    bottom = len(old_image)
    right = len(old_image[0])
    top = 0
    left = 0

  #Checking edge cases
  Col_Count = -num_of_neigh
  Row_Count = -num_of_neigh
  Col_Stop = num_of_neigh
  Row_Stop = num_of_neigh

  x_distance_left = x - left 
  if x_distance_left >= num_of_neigh:
    x_distance_left = num_of_neigh
  x_distance_right = right - x - 1
  if x_distance_right >= num_of_neigh:
    x_distance_right = num_of_neigh
  y_distance_top = y - top
  if y_distance_top >= num_of_neigh:
    y_distance_top = num_of_neigh
  y_distance_bottom = bottom - y - 1
  if y_distance_bottom >= num_of_neigh:
    y_distance_bottom = num_of_neigh

  Col_Count = Col_Count + (num_of_neigh - x_distance_left)
  Row_Count = Row_Count + (num_of_neigh - y_distance_top)
  Col_Stop = Col_Stop - (num_of_neigh - x_distance_right)
  Row_Stop = Row_Stop - (num_of_neigh - y_distance_bottom)

  #Holds the final pixel value for that color
  final_pixel_value = 0

  for Row in range(Row_Count, Row_Stop + 1):
    for Col in range(Col_Count, Col_Stop + 1):
    
      #Summing up all neighboring pixel values based on box blur distribution
      final_pixel_value = final_pixel_value + 1/((num_of_neigh * 2 + 1) ** 2) * old_image[y+Row][x+Col][rgb]

  return final_pixel_value

In [6]:
def blur_area(old_image_path,top_left,bottom_right,num_of_neigh,gauss_or_box,st_dev,ugly_box):
  if type(old_image_path) == str:
    old_image_s = PIL.Image.open(old_image_path)
    old_image = np.asarray(old_image_s)
  else: 
    old_image = old_image_path
  shape = old_image.shape
  height = len(old_image)
  width = len(old_image[0])
  new_image = np.zeros(shape)
  #Getting all the values for each boundary to blur
  top = top_left[1]
  left = top_left[0]
  bottom = bottom_right[1]
  right = bottom_right[0]

  #checking if our coordinates are out of bounds
  if top < 0:
    print("top pixel is out of bounds")
    return 0
  if bottom > height:
    print("bottom pixel is out of bounds")
    return 0
  if left < 0:
    print("left pixel is out of bounds")
    return 0
  if right > width:
    print("right pixel is out of bounds")
    return 0

  #checking if our coordinates make sense
  if top >= bottom:
    print("top pixel must be less than bottom pixel")
    return 0
  if left >= right:
    print("left pixel must be less than right pixel")
    return 0

  #gaussian blur case
  if gauss_or_box == 0:
    #checking if standard deviation is valid
    if st_dev <= 0:
      print("st_dev must be greater than 0")
      return 0
    
    for y in range(height):
      for x in range(width):
        for rgb in range(3):
          #if we are within the range of blurring use gaussian blur, otherwise just use the original pixel
          if y >= top and y <= bottom and x >= left and x <= right:
            new_image[y][x][rgb] = neighbor_values_gauss(x,y,rgb,top_left,bottom_right,num_of_neigh,old_image,st_dev,ugly_box)
          else:
            new_image[y][x][rgb] = int(old_image[y][x][rgb])

    return new_image.astype(int)

  #box blur case
  elif gauss_or_box == 1:
    for y in range(height):
      for x in range(width):
        for rgb in range(3):
          #if we are within the range of blurring use box blur, otherwise just use the original pixel
          if y >= top and y <= bottom and x >= left and x <= right:
            new_image[y][x][rgb] = neighbor_values_box(x,y,rgb,top_left,bottom_right,num_of_neigh,old_image,ugly_box)
          else:
            new_image[y][x][rgb] = int(old_image[y][x][rgb])

    return new_image.astype(int)

  else:
    print("Invalid gauss_or_box number, please choose 0 for gaussian blur and 1 for box blur")

  return 0




In [7]:
def move_part(old_image,old_top_left,old_bottom_right,new_top_left,old_new,next_image,overlay):
  shape = old_image.shape
  height = len(old_image)
  width = len(old_image[0])
  if old_new == 0:
    new_image = np.copy(old_image)
  else:
    new_image = np.copy(next_image)
    

  #Getting all the values for the coordinates to move
  top = old_top_left[1]
  left = old_top_left[0]
  bottom = old_bottom_right[1]
  right = old_bottom_right[0]

  #checking if our coordinates are out of bounds
  if top < 0:
    print("top pixel is out of bounds")
    return 0
  if bottom > height:
    print("bottom pixel is out of bounds")
    return 0
  if left < 0:
    print("left pixel is out of bounds")
    return 0
  if right > width:
    print("right pixel is out of bounds")
    return 0

  #checking if our coordinates make sense
  if top >= bottom:
    print("top pixel must be less than bottom pixel")
    return 0
  if left >= right:
    print("left pixel must be less than right pixel")
    return 0

  #the height and width of the image part to move
  part_height = bottom - top 
  part_width = right - left

  #If our new location is partially out of bounds, we will need to make sure we only access the correct spots
  top_check = 0
  left_check = 0
  bottom_check = 0
  right_check = 0

  if new_top_left[0] < 0:
    left_check = -new_top_left[0]
  elif new_top_left[0] + part_width > width:
    right_check = -(new_top_left[0] + part_width - width)

  if new_top_left[1] < 0:
    top_check = -new_top_left[1]
  elif new_top_left[1] + part_height > height:
    bottom_check = -(new_top_left[1] + part_height - height)

  #moves image, leaves previous spot black
  if overlay == 0:

    #Making previous spot black
    for y in range(top,bottom):
      for x in range(left,right):
        for rgb in range(3):
          new_image[y][x][rgb] = 0

    #Making new spot look like old part
    for y in range(top_check, part_height + bottom_check):
      for x in range(left_check, part_width + right_check):
        for rgb in range(3):
          new_image[y + new_top_left[1]][x + new_top_left[0]][rgb] = old_image[top + top_check + y][left + left_check + x][rgb]

    return new_image

  #moves image, leaves previous spot same as before, but with original on top
  elif overlay == 1:

    #Making new spot look like old part
    for y in range(top_check, part_height + bottom_check):
      for x in range(left_check, part_width + right_check):
        for rgb in range(3):
          new_image[y + new_top_left[1]][x + new_top_left[0]][rgb] = old_image[top + top_check + y][left + left_check + x][rgb]

    #Putting the original part back, in case it has been overlapped
    for y in range(top,bottom):
      for x in range(left,right):
        for rgb in range(3):
          new_image[y][x][rgb] = old_image[y][x][rgb]

    return new_image

  #moves image, leaves previous spot same as before, but with new on top
  elif overlay == 2:

    #Putting the original part back, in case it has been overlapped
    for y in range(top,bottom):
      for x in range(left,right):
        for rgb in range(3):
          new_image[y][x][rgb] = old_image[y][x][rgb]

    #Places old image onto new spot regardless of what was there before
    for y in range(top_check, part_height + bottom_check):
      for x in range(left_check, part_width + right_check):
        for rgb in range(3):
          new_image[y + new_top_left[1]][x + new_top_left[0]][rgb] = old_image[top + top_check + y][left + left_check + x][rgb]

    return new_image

In [74]:
def motion_blur(frame1_path, boxes1, id1, frame2_path, boxes2, id2, blend_weight, fps):

  #read the image
  frame1 = cv2.imread(frame1_path, cv2.IMREAD_COLOR)
  frame2 = cv2.imread(frame2_path, cv2.IMREAD_COLOR)
  top = len(frame1)
  left = len(frame1[0])
  right = 0 
  bottom = 0
  check = 0
  after_frame = frame1
  #check if the frames have ids that match
  for j in range(len(id1)):
    if id1[j] in id2:
      check = 1
      x = np.where(id2 == id1[j])[0][0]
     
      frame_a_top_left = (int(boxes1[j][0]), int(boxes1[j][1]))
      frame_a_bottom_right = (int(boxes1[j][2]), int(boxes1[j][3]))
      frame_b_top_left = (int(boxes2[x][0]), int(boxes2[x][1]))
      frame_b_bottom_right = (int(boxes2[x][2]), int(boxes2[x][3]))

      #direction vector the box is moving in, only for detecting translation, not rotation. positive x = right, positive y = up
      direction = (frame_b_top_left[0]-frame_a_top_left[0], frame_a_top_left[1]-frame_b_top_left[1])
      
      #distance for calculating speed
      distance = math.sqrt(((direction[0] * 5 / frame1.shape[1]) ** 2) + ((direction[1] * 5 / frame1.shape[0]) ** 2))

      #time between frames in 1/fps, so speed is just distance times the frames
      speed = distance * fps

      if speed < 1:
        speed = 1
      # Specify the kernel size.
      # The greater the size, the more the motion.
      kernel_size = int(speed)

      # Create the kernel based on the direction.
      kernel_direction = np.zeros((kernel_size, kernel_size))

      if direction[1] == 0 :
        # Fill the horizontal middle row with ones.
        kernel_direction[:, int((kernel_size - 1)/2)] = np.ones(kernel_size)
      elif direction[0]  == 0:
        # Vertical middle row
        kernel_direction[int((kernel_size - 1)/2), :] = np.ones(kernel_size)
      elif direction[1]/direction[0] < 0:
        # Top-left diagonal 
        np.fill_diagonal(kernel_direction, 1)
      else: 
        # Top-right diagonal
        np.fill_diagonal(np.fliplr(kernel_direction), 1)

      # Normalize.
      kernel_direction /= kernel_size
   
      filter = cv2.filter2D(frame1, -1, kernel_direction)

      new_top_left = (int((frame_a_top_left[0] + frame_b_top_left[0])/2), 
                      int((frame_a_top_left[1] + frame_b_top_left[1])/2)) 
      
      frame_new = move_part(filter, frame_a_top_left, frame_a_bottom_right, new_top_left, 1, frame1, 2)

      #frame_blur1 = blur_area(filter,frame1_top_left,frame1_bottom_right,3,0,2,1)
      #frame_blur2 = blur_area(frame_new,frame1_top_left,frame1_bottom_right,3,0,2,1)

      #blend the images together
      blend = cv2.addWeighted(frame1, blend_weight, frame_new, 1-blend_weight, 0.0)

      top_left = (min(new_top_left[0],frame_a_top_left[0]),min(new_top_left[1],frame_a_top_left[1]))
      bottom_right = (min(max(new_top_left[0] + frame_a_bottom_right[0] - frame_a_top_left[0],
                          frame_a_bottom_right[0]),len(frame1[0])),
                      min(max(new_top_left[1] + frame_a_bottom_right[1] - frame_a_top_left[1],
                          frame_a_bottom_right[1]),len(frame1)))

      if top_left[0] < left:
        left = top_left[0]
      if top_left[1] < top:
        top = top_left[1]
      if bottom_right[0] > right:
        right = bottom_right[0]
      if bottom_right[1] > bottom:
        bottom = bottom_right[1]
                        
      before_frame = move_part(blend, top_left, bottom_right, top_left, 1, frame1, 2)

      if j >= 1:
        after_frame = move_part(before_frame, top_left, bottom_right, top_left, 1, after_frame, 2)
      else: 
        after_frame = before_frame

  if check == 1:
    return (after_frame,1)
  else:
    return (frame1,0)



In [9]:
def frames_to_array(pathIn,frame_array):
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    height, width, _ = cv2.imread(pathIn + '/' + files[0]).shape

    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    for i in range(len(files)):

        filename=pathIn + '/' + files[i]
        #reading each files
        img = cv2.imread(filename)    
        height, width, layers = img.shape
        size = (width,height)
        #print(filename)
        #inserting the frames into an image array
        frame_array.append(img)

    return size

In [10]:
def add_frames_to_video(image,size,frame_number,frames,frame_array):
    
    width = size[1]
    height = size[0]
    new_img_resize = np.resize(image,(width,height,3))
    #print('Image width resized to: ',width)
    #print('Image height resized to: ',height)
    new_img_final = np.asarray(new_img_resize)

    for x in range(frames):
        frame_array.insert(frame_number + x, new_img_final)
        #print('Image inserted at frame: ',frame_number + x)
          

In [11]:
def convert_frames_to_video(pathOut,fps,frame_array, size):
  
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [12]:
def controller(boxes, id, path_to_video, path_for_frames, path_to_new_video):

  #convert video to frames and saves fps
  vidcap = cv2.VideoCapture(path_to_video) #put path to video
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  success,image = vidcap.read()
  count = 0

  while success:
    cv2.imwrite(f"{path_for_frames}/frame{count}.jpg", image)     # save frame as JPEG file      
    success,image = vidcap.read()
    
    #print('Read a new frame: ', success)
    count += 1

  duration = count / fps
  #frame_array will store all the frames as a numpy array, and will allow us to insert frames where we want to
  frame_array = [] 
  #save the size, and add the original video frames into the array
  size = frames_to_array(path_for_frames, frame_array)
  #keeps track of how many frames we added, for offset in the array
  frames_added = 0

  #count = amount of frames we have.
  for i in range(count - 1):

      blurred = motion_blur(f"{path_for_frames}/frame{i}.jpg",boxes[i],id[i],f"{path_for_frames}/frame{i+1}.jpg",
                            boxes[i+1],id[i+1], 0.3, fps)
      
      
      #add blurred image between frames
      add_frames_to_video(blurred[0],size,i+1+frames_added,1,frame_array) #200 for testing purposes, change later
      #keep track of offset needed due to additional frames.
      frames_added += 1 #change this too

  convert_frames_to_video(f"{path_to_new_video}/output4.avi", int(len(frame_array)/duration), frame_array, size)
  return (frame_array)
   